In [ ]:
import polars as pl
import networkx as nx
import time
import rtsvg
from linknode_graph_patterns import LinkNodeGraphPatterns
rt = rtsvg.RACETrack()
_patterns_ = LinkNodeGraphPatterns()

In [ ]:
def dims(pos): return [min(pos[k][0] for k in pos), max(pos[k][0] for k in pos), min(pos[k][1] for k in pos), max(pos[k][1] for k in pos)]
def dimsPrintable(pos):
    _arr_ = dims(pos)
    return f'({_arr_[0]:.2f}, {_arr_[1]:.2f}), ({_arr_[2]:.2f}, {_arr_[3]:.2f})'

_tiles_ = []

for _graph_num_ in _patterns_.stanford_facebook_graph_nums:
    _type_ = 'stanford_facebook_networks'
    g  = _patterns_.createPattern(_type_, graph_num=_graph_num_)
    df = _patterns_.nxGraphToPolarsDataFrame(g)

    # Community colors
    _node_colors_, _community_index_ = {}, 0
    for _community_ in nx.community.louvain_communities(g):
        for _node_ in _community_: _node_colors_[_node_] = str(_community_index_)
        _community_index_ += 1

    params = {'df':df, 'relationships':[('fm','to')], 'w':256, 'h':256, 'node_size':'small', 'link_arrow':False, 'node_color':_node_colors_}
    
    t0 = time.time()
    pos_nx = nx.spring_layout(g)
    t1 = time.time()
    _tiles_.append(rt.titleSVG(rt.link(pos=pos_nx,  **params), f'{_type_} Spring (NX) {t1-t0:.2f}s'))

    t0 = time.time()
    lmds    = rtsvg.LandmarkMDSLayout(g)
    pos_lmds = lmds.results()
    t1 = time.time()
    _tiles_.append(rt.titleSVG(rt.link(pos=pos_lmds, **params), f'Landmark MDS {t1-t0:.2f}s'))

    t0 = time.time()
    pmds     = rtsvg.PivotMDSLayout(g)
    pos_pmds = pmds.results()
    t1 = time.time()
    _tiles_.append(rt.titleSVG(rt.link(pos=pos_pmds, **params), f'Pivot MDS {t1-t0:.2f}s'))

    # print(dimsPrintable(pos_nx), dimsPrintable(pos_lmds), dimsPrintable(pos_pmds))

rt.table(_tiles_, per_row=3)

In [ ]:
_tiles_    = []
for i in range(len(_patterns_)):
    _type_  = _patterns_[i]
    if _type_ == 'stanford_facebook_networks': continue
    g       = _patterns_.createPattern(_type_)
    _dists_ =  dict(nx.all_pairs_dijkstra_path_length(g))
    _lms_  = rt.identifyLandmarks(g)
    _node_colors_ = {}
    for _lm_ in _lms_[0]: _node_colors_[_lm_] = 'red'
    df     = _patterns_.nxGraphToPolarsDataFrame(g)
    params = {'df':df, 'relationships':[('fm','to')], 'w':256, 'h':256, 'node_size':'small', 'link_arrow':False, 'node_color':_node_colors_}
    
    t0 = time.time()
    pos_nx = nx.spring_layout(g)
    t1 = time.time()
    _tiles_.append(rt.titleSVG(rt.link(pos=pos_nx,  **params), f'{_type_} Spring (NX) {t1-t0:.2f}s'))
    
    t0 = time.time()
    pos_ref = rt.springLayout(g)
    t1 = time.time()
    _tiles_.append(rt.titleSVG(rt.link(pos=pos_ref, **params), f'Spring {t1-t0:.2f}s'))

    t0 = time.time()
    pfdl_k0    = rtsvg.PolarsForceDirectedLayout(g, k=0)
    pos_psl_k0 = pfdl_k0.results()
    t1 = time.time()
    _tiles_.append(rt.titleSVG(rt.link(pos=pos_psl_k0, **params), f'Polar_K_0 {t1-t0:.2f}s'))
                   
    t0 = time.time()
    pfdl_k1    = rtsvg.PolarsForceDirectedLayout(g, k=1)
    pos_psl_k1 = pfdl_k1.results()
    t1 = time.time()
    _tiles_.append(rt.titleSVG(rt.link(pos=pos_psl_k1, **params), f'Polar_K_1 {t1-t0:.2f}s'))

    t0 = time.time()
    cpl_k0    = rtsvg.ConveyProximityLayout(g, use_resistive_distances=False, k=0)
    pos_cpl_k0 = cpl_k0.results()
    t1 = time.time()
    _tiles_.append(rt.titleSVG(rt.link(pos=pos_cpl_k0, **params), f'CPL_K_0 {t1-t0:.2f}s'))

    t0 = time.time()
    cpl_k1    = rtsvg.ConveyProximityLayout(g, use_resistive_distances=False, k=1)
    pos_cpl_k1 = cpl_k1.results()
    t1 = time.time()
    _tiles_.append(rt.titleSVG(rt.link(pos=pos_cpl_k1, **params), f'CPL_K_1 {t1-t0:.2f}s'))

    t0 = time.time()
    lmds    = rtsvg.LandmarkMDSLayout(g)
    pos_lmds = lmds.results()
    t1 = time.time()
    _tiles_.append(rt.titleSVG(rt.link(pos=pos_lmds, **params), f'Landmark MDS {t1-t0:.2f}s'))

    t0 = time.time()
    pmds     = rtsvg.PivotMDSLayout(g)
    pos_pmds = pmds.results()
    t1 = time.time()
    _tiles_.append(rt.titleSVG(rt.link(pos=pos_pmds, **params), f'Pivot MDS {t1-t0:.2f}s'))

rt.table(_tiles_, per_row=8, spacer=10)